In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import h5py
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas as pd
from scipy.special import softmax 
from tqdm import tqdm


In [3]:
# path = "F:/git/eeg_prep/processed_files/039_000500_000500.hdf5"
path = "/media/dan/Data/git/eeg_prep/processed_files/098_000500_000500.hdf5"
with h5py.File(path, "r") as f:
    A_mats_500 = f['data']['A_mats'][:]
    A_mats_500_mask = f['data']['A_mask'][:]
    soz = f['metadata']["patient_info"]['soz'][:].astype(bool)
    chnames = f['metadata']['channels'][:]

In [ ]:
# Simple embedding model
class EmbeddingNet(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim=256):
        super(EmbeddingNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.fc1 = nn.Linear(input_dim, self.hidden_dim)
        self.fc2 = nn.Linear(self.hidden_dim, embedding_dim)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)  # Embedding output (no activation for embeddings)
        return x

In [4]:
A = A_mats_500[:,:,A_mats_500_mask==1]
A = np.abs(A)
eye = np.eye(A.shape[0]) == 1
A[eye,:] = np.nan
# normalize 0-1 for each row so each row at each time point is 0-1 
for time in range(A.shape[-1]):
    # Get the 2D matrix at the specific time step
    slice_t = A[:, :, time]
    
    # Normalize each row individually
    row_min = np.nanmin(slice_t, axis=1, keepdims=True)
    row_max = np.nanmax(slice_t, axis=1, keepdims=True)
    
    # Avoid division by zero in case of zero-variance rows
    normalized_slice = (slice_t - row_min) / (row_max - row_min + 1e-10)
    
    # Place the normalized slice back into A
    A[:, :, time] = normalized_slice
A.shape

(140, 140, 542)

In [ ]:

# Initialize model, loss, and optimizer
input_dim = X.shape[1]  # Dimension of each vector (C)
embedding_dim = 2       # Low-dimensional embedding for clustering

model = EmbeddingNet(input_dim, embedding_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
